In [ ]:
import os
import torch
import pandas as pd
import torchvision.transforms as TF
from torch.utils.data import Dataset, DataLoader

# Define global variables:
samples_path = os.environ.get('SAMPLES_PATH') or '.\\samples'

x_data_file = os.environ.get('X_DATA_FILE') or 'jupyter_landmarks.csv'
x_cols_start_index = os.environ.get('X_COLS_START_INDEX') or 1
x_cols_end_index = os.environ.get('X_COLS_END_INDEX') or 209

y_data_file = os.environ.get('Y_DATA_FILE') or 'unity_blendshapes.csv'
y_cols_start_index = os.environ.get('Y_COLS_START_INDEX') or 2
y_cols_end_index = os.environ.get('Y_COLS_END_INDEX') or 74


In [ ]:
def getDataFromCSV(env, file, start, end):
    return pd.read_csv(f'{samples_path}\\{env}\\{file}',
                               usecols = range(start, end))


In [ ]:
def normallizeData(data):
    for i in range(data.shape[1]):
        norm = torch.std(data[:, i])
        norm = 0.001 if norm == 0 else norm
        data[:, i] = (data[:, i] - torch.mean(data[:, i])) / norm
        
    return data 

In [ ]:
def getData(env):
    # Get data from csv files
    x = getDataFromCSV(env, x_data_file, x_cols_start_index, x_cols_end_index)
    y = getDataFromCSV(env, y_data_file, y_cols_start_index, y_cols_end_index)

    # Transforms the data to tensors
    x_tensor = torch.tensor(x.values, requires_grad=True).float()
    y_tensor = torch.tensor(y.values, requires_grad=True).float()

    # Normallize the data
    norm_x_tensor = normallizeData(x_tensor)
    norm_y_tensor = normallizeData(y_tensor)

    return norm_x_tensor, norm_y_tensor

In [ ]:
class LandmarksDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y


    def __getitem__(self, index):
        return (self.x[index], self.y[index])
    
    def __len__(self):
        return len(self.x)

In [ ]:
def getDataset(env):
    x, y = getData(env)
    return LandmarksDataset(x, y)

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import pandas as pd


def create_nn(batch_size=50, learning_rate=0.001, epochs=15,
              log_interval=10):
    
    train_dataset = getDataset('train')
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size,
        shuffle=True)

    test_dataset=getDataset('test')
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size,
        shuffle=False)

    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(208, 1024)
            self.fc2 = nn.Linear(1024, 512)
            self.fc3 = nn.Linear(512, 256)
            self.fc4 = nn.Linear(256, 128)
            self.fc5 = nn.Linear(128, 72)
            
        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            x = F.relu(self.fc4(x))
            x = self.fc5(x)
            return x

    net = Net()

    # create a stochastic gradient descent optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    # create a loss function
    # criterion = nn.L1Loss(reduction='mean')
    criterion = nn.MSELoss(reduction='mean')

    # run the main training loop
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data), Variable(target)
            optimizer.zero_grad()
            net_out = net(data)
            loss = criterion(net_out, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.item()))

    # run a test loop
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        net_out = net(data)

        # sum up batch loss
        test_loss += criterion(net_out, target).item()
#         pred = net_out.data.max(1)[1]  # get the index of the max log-probability
#         correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}'.format(
        test_loss))

if __name__ == "__main__":
    run_opt = 2
    if run_opt == 1:
        simple_gradient()
    elif run_opt == 2:
        create_nn()